# Inspect, Crop/Tile, and Export UAS Imagery

To create the environment, install the following
```
conda install -c conda-forge jupyter rasterio pyproj pillow matplotlib
``` 

### Set up the python environment and prep some variables

In [ ]:
#Import librarries
from PIL import Image
import os
import argparse
import numpy as np
import json
import csv
import rasterio
import matplotlib
import folium
#from pyproj import Proj, transform
from pyproj import Transformer

%matplotlib inline

Image.MAX_IMAGE_PIXELS = None

In [ ]:
#Filename for the image file
orthomosaic_file = './data/raw/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1.tif'

# generous estimate of object length in meters
seal_length = 2.6

# nodata value in your imagery, if one applies for your input dataset
nodata_value = (255,255,255)

In [ ]:
#Read in the image file
dataset = rasterio.open(orthomosaic_file)

### Examine the metadata of the orthomosiac
This is mostly for checking/viewing info, but it does also set up some critical variables for later

In [ ]:
# what is the name of this image
print('Image filename: {n}\n'.format(n=dataset.name))

# How many bands does this image have?
num_bands = dataset.count
print('Number of bands in image: {n}\n'.format(n=num_bands))

# How many rows and columns?
rows, cols = dataset.shape
print('Image size is: {r} rows x {c} columns\n'.format(r=rows, c=cols))

# Does the raster have a description or metadata?
desc = dataset.descriptions
metadata = dataset.meta

if desc[0]: 
    print('Raster description: {desc}\n'.format(desc=desc))

# What driver was used to open the raster?
driver = dataset.driver
print('Raster driver: {d}\n'.format(d=driver))

# What is the raster's projection?
proj = dataset.crs
print('Image projection:')
print(proj, '\n')

# What is the raster's "geo-transform"
gt = dataset.transform
print('Image geo-transform:\n{gt}\n'.format(gt=gt))

# What are the pixel dimensions
pixelSizeX = gt[0]
pixelSizeY =-gt[4]
print("pixel X dimension: " + str(pixelSizeX))
print("pixel Y dimension: " + str(pixelSizeY))

#print('All raster metadata:')
#print(metadata)
#print('\n')

### Plot the image
Also mostly for viewing and confirming that things look appropriate

In [ ]:
from rasterio.plot import show
from rasterio.windows import Window
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15,15))

show(dataset.read((1,2,3), window=Window(5000, 5000, 2000, 2000)), transform=dataset.transform, ax=ax)
plt.show()

### Visualize on the Map
Again, mostly a disgnostic check

In [ ]:
#Rasterio transformations
utm_tl = dataset.transform * (0, 0)                          #Top left
utm_br = dataset.transform * (dataset.width, dataset.height) #Bottom right
utm_center = dataset.transform * (dataset.width // 2, dataset.height // 2)

#Create the tranformer object
transformer = Transformer.from_crs(crs_from = dataset.crs.to_epsg(), crs_to = "epsg:4326") 

#Create set of absolute coordinates
positions = (
    dataset.transform * (0, 0), 
    dataset.transform * (dataset.width, 0), 
    dataset.transform * (dataset.width, dataset.height), 
    dataset.transform * (0, dataset.height))

#Transform coordinates to WGS 84
tl_lat, tl_long = transformer.transform(utm_tl[0], utm_tl[1])
br_lat, br_long = transformer.transform(utm_br[0], utm_br[1])
center_lat, center_long = transformer.transform(utm_center[0], utm_center[1])

lats, longs = transformer.transform(np.array(positions)[:,0],np.array(positions)[:,1])
points = list(zip(lats, longs))
print(tl_long,tl_lat)

In [ ]:
m = folium.Map(location=[center_lat, center_long])

tooltip="Raster"
#folium.Marker([tl_lat, tl_long], popup='<i>Raster Top Left</i>', tooltip=tooltip).add_to(m)
#folium.Marker([br_lat, br_long], popup='<i>Raster Bottom right</i>', tooltip=tooltip).add_to(m)
#folium.Marker([center_lat, center_long], popup='<i>Raster Center</i>', tooltip=tooltip).add_to(m)

folium.Polygon(points, 
               tooltip=tooltip, 
               popup='Laurelhurst Park',
               color='#3186cc',
               fill=True,
               fill_color='#3186cc').add_to(m)

#folium.PolyLine(points, color="red", weight=100, opacity=1).add_to(m)

m

### Crop the Image into Tiles
This section breaks up the orthomosaic image into tiles. <b> It is important to set the `overlap` variable to the pixel-length of at least 1 target object (here, a seal) </b> in case the tiling process "cuts up" some of your objects into undetectable shapes at the edges. Adequate `overlap` ensures that a target object that gets "cut up" on one edge, will be intact in an adjacent image.

In [ ]:
from rasterio.plot import reshape_as_image
from rasterio.windows import Window

# set tile size (pixels): this is dictated by what constitutes a managable filesize for processing
tile_height = tile_width = 2000

# set overlap: this should equal 1–2x the pixel-length of our feature of interest
overlap = round((seal_length / pixelSizeX) * 1.1)

stride = tile_height - overlap
start_num=0

def crop(orthomosaic_file, tile_height, tile_width, stride, img_dict, prj_name):
    im = rasterio.open(orthomosaic_file) 
    img_height, img_width = im.shape
    print(im.shape)
    count = 0
    for r in range(0, img_height, stride):
        for c in range(0, img_width, stride):
            tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
            tile = reshape_as_image(tile)
            top_pixel = [c,r]
            count += 1
            yield tile, top_pixel

### Split the image up into `height` × `width` patches

In [ ]:
orthomosaic_file.split("/")[-1].split(".")[0]

In [ ]:
prj_name = orthomosaic_file.split("/")[-1].split(".")[0]
img = Image
img_dict = {}

# use this variable to set output directory
output_dir = 'tiled_data'
try:
  os.makedirs(output_dir)
except: pass

# create the dir if it doesn't already exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# break it up into crops
for k, tile_w_point in enumerate(crop(orthomosaic_file, tile_height, tile_width, stride, img_dict, prj_name), start_num):
    empty_data = [(0,0,0), nodata_value]
    try:
      img=Image.fromarray(tile_w_point[0])
    except ValueError:
      print("End of set at file " + image_name)
      break
    image_name = prj_name + "---%s.png" % k
    print(image_name)
    corner1, corner2, corner3, corner4 = img.load()[0, 0], img.load()[0, img.size[1]-1], img.load()[img.size[0]-1, img.size[1]-1], img.load()[img.size[0]-1, 0]
    if corner1 in empty_data and corner2 in empty_data and corner3 in empty_data and corner4 in empty_data:
      print("empty tile, skipped")
      continue
    img_dict[image_name] = tile_w_point[1]
    path=os.path.join(output_dir, image_name)
    img.save(path)

### Create a .json file with all image names and geospatial metadata
This is important for storing how the tiles fit together, we will need this later to stitch our detections back together

In [ ]:
full_dict = {"image_name" : orthomosaic_file,
            "image_locations" : img_dict,
             "crs" : str(dataset.crs)
            }
json_output = output_dir + '/tiling_scheme.json'

with open(json_output, 'w') as fp:
    json.dump({"orthomosaic_file":orthomosaic_file.split("/")[-1], "spatial_reference":str(proj), "transform":gt, "tile_height":tile_height, "tile_width":tile_width, "tile_overlap":overlap, "tile_pointers":full_dict}, fp)

##### At the end of this script you should have downloaded all tile files and the `tiling_scheme.json` files. The tiles set should be ready for annotation in VIA to create training data. When you load the script to train the CNN, you will need the tiles and `tiling_scheme.json` file from this script (+ the `json` file from VIA, + annotation `csv` files that have been converted to RetinaNet format)

Next steps:

2) create annotations in VIA, save `csv` output

3) convert annotations from VIA format to RetinaNet format, with Training, Testing, and Validation subsets

4) train, refine, and test CNN using VIA annotations and the tiles generated here, produce precision metrics

5) export CNN outputs, manual annotations, and tile footprints as shapefiles
